In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense,LSTM
from tensorflow.keras import layers, models

In [2]:
df=pd.read_csv("Ankleshwar.csv")
df=df.iloc[:,1:]
df

,Date,pm25
0,04-02-2019 19:00,70.52
1,04-02-2019 20:00,61.39
2,04-02-2019 21:00,59.94
3,04-02-2019 22:00,76.44
4,04-02-2019 23:00,98.54
...,...,...
33531,02-12-2022 22:00,115.37
33532,02-12-2022 23:00,104.06
33533,03-12-2022 00:00,110.59
33534,03-12-2022 01:00,122.87


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import layers, models

# --------------------------------------------------
# STEP 1: Data Preparation
# --------------------------------------------------


# Normalize PM2.5 column
scaler = MinMaxScaler()
df['pm25_scaled'] = scaler.fit_transform(df[['pm25']])

# Create sequences for training
def create_sequences(data, window=24):
    X, y = [], []
    for i in range(len(data) - window):
        if not np.isnan(data[i + window]):
            seq_x = data[i:i+window]
            if not np.any(np.isnan(seq_x)):
                X.append(seq_x)
                y.append(data[i + window])
    return np.array(X), np.array(y)

pm25_array = df['pm25_scaled'].values
window = 24
X, y = create_sequences(pm25_array, window)
X = X.reshape((X.shape[0], window, 1))  # [samples, timesteps, features]

# --------------------------------------------------
# STEP 2: Build BiLSTM Model
# --------------------------------------------------

model = models.Sequential([
    layers.Bidirectional(layers.LSTM(64, return_sequences=False), input_shape=(window, 1)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=25, batch_size=32, verbose=1)

# --------------------------------------------------
# STEP 3: Impute Missing Values
# --------------------------------------------------

filled = df['pm25_scaled'].copy()
for i in range(window, len(filled)):
    if np.isnan(filled[i]):
        context = filled[i - window:i]
        if not np.any(np.isnan(context)):
            input_seq = np.array(context).reshape((1, window, 1))
            prediction = model.predict(input_seq, verbose=0)[0, 0]
            filled[i] = prediction

# --------------------------------------------------
# STEP 4: Inverse Transform and Save
# --------------------------------------------------

df['pm25_bilstm'] = scaler.inverse_transform(np.array(filled).reshape(-1, 1))
df.to_csv('data_bilstm.csv', index=False)

# Optional Preview
print(df[['datetime', 'pm25', 'pm25_bilstm']].head())

C:\Users\Subham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 86s 40ms/step - loss: 3.8642e-04
Epoch 2/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 25s 39ms/step - loss: 1.7788e-04
Epoch 3/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 26s 40ms/step - loss: 2.0599e-04
Epoch 4/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 26s 39ms/step - loss: 1.4759e-04
Epoch 5/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 26s 40ms/step - loss: 2.0091e-04
Epoch 6/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 27s 43ms/step - loss: 1.6477e-04
Epoch 7/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - loss: 1.6225e-04
Epoch 8/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - loss: 1.9195e-04
Epoch 9/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 21s 32ms/step - loss: 1.2618e-04
Epoch 10/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 23s 37ms/step - loss: 1.3035e-04
Epoch 11/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 23s 36ms/step - loss: 1.5163e-04
Epoch 12/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 19s 29ms/step - loss: 1.2644e-04
Epoch 13/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 19s 30ms/step - loss: 1.9687e-04
Epoch 14/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 19s 29

In [ ]:
df.to_csv("biLSTM.csv")